# Coding Homework - Week 2

## Name: **Hong Pengfei**
## Student ID: **1002949**

In [1]:
from search import *
from functools import partial

## Problem 1

In [2]:
WORDS = set(i.lower().strip() for i in open('words2.txt'))
def is_valid_word(word): return word in WORDS
def change_string(old_string, character, index): return old_string[:index] + character + old_string[index+1:]

In [3]:
class WordLadders(Problem):
    
    def __init__(self, initial, goal):
        super().__init__(initial, goal)
        
    def actions(self, state):
        action_list = []
        for i in range(len(state)):
            for l in string.ascii_lowercase:
                changed_string = change_string(state, l, i)
                if is_valid_word(changed_string):
                    action_list.append(f"Change {state} with {changed_string}")
                    
        return action_list
    
    def result(self, state, action):
        return action.split()[-1]

In [4]:
wl = WordLadders("cars", "cats")
node = breadth_first_search(wl)
# node = uniform_cost_search(wl)
node.solution()

['Change cars with cats']

In [5]:
wl = WordLadders("cold", "warm")
node = breadth_first_search(wl)
node.solution()

['Change cold with cord',
 'Change cord with word',
 'Change word with ward',
 'Change ward with warm']

In [6]:
wl = WordLadders("best", "math")
node = breadth_first_search(wl)
node.solution()

['Change best with bast',
 'Change bast with mast',
 'Change mast with mash',
 'Change mash with math']

## Problem 2

### Part 1: state
- Current City and current time is a good choice of state

### Part 2:

In [7]:
class Flight:
    def __init__(self, start_city, start_time, end_city, end_time):
        self.start_city = start_city
        self.start_time = start_time
        self.end_city = end_city
        self.end_time = end_time
    
    def __str__(self):
        return str((self.start_city, self.start_time)) + ' -> ' + str((self.end_city, self.end_time))
    __repr__ = __str__
    
    def contained(self, inputs):
        city, time = inputs
        return city == self.start_city and time <= self.start_time
    
flightDB = [
    Flight('Rome', 1, 'Paris', 4),
    Flight('Rome', 3, 'Madrid', 5),
    Flight('Rome', 5, 'Istanbul', 10),
    Flight('Paris', 2, 'London', 4),
    Flight('Paris', 5, 'Oslo', 7),
    Flight('Paris', 5, 'Istanbul', 9),
    Flight('Madrid', 7, 'Rabat', 10),
    Flight('Madrid', 8, 'London', 10),
    Flight('Istanbul', 10, 'Constantinople', 10)
]

print("Flights")
for f in flightDB:
    print(f)

Flights
('Rome', 1) -> ('Paris', 4)
('Rome', 3) -> ('Madrid', 5)
('Rome', 5) -> ('Istanbul', 10)
('Paris', 2) -> ('London', 4)
('Paris', 5) -> ('Oslo', 7)
('Paris', 5) -> ('Istanbul', 9)
('Madrid', 7) -> ('Rabat', 10)
('Madrid', 8) -> ('London', 10)
('Istanbul', 10) -> ('Constantinople', 10)


### Part 3

In [8]:
class Itinerary(Problem):
    
    def __init__(self, initial, goal):
        super().__init__(initial, goal)
        
    def actions(self, state):
        action_list = []
        for f in flightDB:
            if f.contained(state):
                action_list.append(f)
        return action_list
    
    def result(self, state, action):
        return (action.end_city, action.end_time)
    
    def goal_test(self, state):
        current_city, current_time = state
        goal_city, goal_time = self.goal
        return current_city == goal_city and current_time <= goal_time
    
def find_itinerary(start_city, start_time, end_city, deadline):
    itinerary = Itinerary( (start_city,start_time), (end_city,deadline) )
    node = breadth_first_search(itinerary)
    if not node: return None
    return node.solution()

In [9]:
find_itinerary('Rome', 0, 'Constantinople', 10)

[('Rome', 5) -> ('Istanbul', 10), ('Istanbul', 10) -> ('Constantinople', 10)]

### Part 4
- Ben's solution will work assuming the time is discrete. it will take around  $2x$ calls

### Normal Solution

In [10]:
def find_shortest_itinerary(start_city, end_city):
    start_time = 1
    deadline = 1
    solution = None
    while not solution:
        solution = find_itinerary(start_city, start_time, end_city, deadline)
        deadline += 1
    return solution

In [11]:
find_shortest_itinerary('Rome', 'Constantinople')

[('Rome', 5) -> ('Istanbul', 10), ('Istanbul', 10) -> ('Constantinople', 10)]

### optimized solution: time complexity should be O(logn)

In [12]:
def find_shortest_itinerary(start_city, end_city):
    start_time = 1
    deadline = 1
    sol = None
    while not sol:
        sol = find_itinerary(start_city, start_time, end_city, deadline)
        deadline *= 2
    
    deadline_l = deadline // 2
    deadline_h = deadline
    deadline = (deadline_h + deadline_l) // 2
    while deadline_h > deadline_l:
        sol = find_itinerary(start_city, start_time, end_city, deadline)
        if not sol:
            deadline_h = deadline - 1
        else:
            deadline_l = deadline + 1
        deadline = (deadline_h + deadline_l) // 2
        
    return sol

In [13]:
find_shortest_itinerary('Rome', 'Constantinople')

[('Rome', 5) -> ('Istanbul', 10), ('Istanbul', 10) -> ('Constantinople', 10)]